In [1]:
# Import Libraries
import pandas as pd
import re

In [2]:
#Load Dataset
df = pd.read_excel('PAN Dataset.xlsx')

In [3]:
print(df.head())

  Pan_Numbers
0  VGLOD3180G
1  PHOXD7232L
2  MGEPH6532A
3  JJCHK4574O
4  XTQIJ2330L


In [7]:
print(df.shape)

(10000, 1)


In [9]:
total_records = len(df)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Pan_Numbers  9035 non-null   object
dtypes: object(1)
memory usage: 78.3+ KB


In [13]:
# Data Cleaning

In [15]:
df['Pan_Numbers'] = df['Pan_Numbers'].str.strip().str.upper()

In [17]:
df.head()

,Pan_Numbers
0,VGLOD3180G
1,PHOXD7232L
2,MGEPH6532A
3,JJCHK4574O
4,XTQIJ2330L


In [19]:
df[df['Pan_Numbers'] == '']

,Pan_Numbers
5019,
5020,


In [21]:
df['Pan_Numbers'].isna().sum()

965

In [23]:
df = df.replace({'Pan_Numbers', ''}, pd.NA)

In [25]:
df[df['Pan_Numbers'] == '']

,Pan_Numbers


In [27]:
df['Pan_Numbers'].isna().sum()

967

In [29]:
df = df.dropna(subset = 'Pan_Numbers')

In [31]:
df.shape

(9033, 1)

In [33]:
df['Pan_Numbers'].nunique()

9025

In [35]:
df = df.drop_duplicates(subset = 'Pan_Numbers', keep = 'first')

In [37]:
df.duplicated().sum()

0

In [39]:
# Data Validation

In [43]:
def has_adjacent_repetition(pan):
    for i in range(len(pan) - 1):
        if pan[i] == pan[i + 1]:
            return True
    return False

def is_sequential(pan):
    for i in range(len(pan)-1):
        if ord(pan[i + 1]) - ord(pan[i]) != 1:
            return False
    return True

def is_valid_pan(pan):
    if len(pan) != 10:
        return False

    if not re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$', pan):
        return False

    if has_adjacent_repetition(pan):
        return False

    if is_sequential(pan[:5]):
        return False

    if is_sequential(pan[5:9]):
        return False

    return True

df['Status'] = df['Pan_Numbers'].apply(lambda x: 'Valid' if is_valid_pan(x) else 'Invalid')

In [45]:
df.head()

,Pan_Numbers,Status
0,VGLOD3180G,Valid
1,PHOXD7232L,Valid
2,MGEPH6532A,Valid
3,JJCHK4574O,Invalid
4,XTQIJ2330L,Invalid


In [47]:
valid_cnt = (df['Status'] == 'Valid').sum()
Invalid_cnt = (df['Status'] == 'Invalid').sum()
missing_cnt = total_records - (valid_cnt + Invalid_cnt)

print(total_records)
print(valid_cnt)
print(Invalid_cnt)
print(missing_cnt)

10000
3186
5839
975


In [49]:
df_summary = pd.DataFrame({
    'Total Processed Records':[total_records],
    'Total Valid PAN':[valid_cnt],
    'Total Invalid PAN':[Invalid_cnt],
    'Total Missing PAN':[missing_cnt]
})

In [51]:
with pd.ExcelWriter('PAN Validation Result.xlsx') as writer:
    df.to_excel(writer, sheet_name = 'PAN Validations', index = False)
    df_summary.to_excel(writer, sheet_name = 'Summary', index = False)